In [ ]:
from urllib.request import urlopen
import json
import pandas as pd

KEYs = ['key1', 'key2', 'key3', 'key4', 'key5', 'key6']
key_index = 0

In [ ]:
from os.path import exists

# Lendo csv com os dados do minuto anterior para fins de comparação
if (exists('evo_last_data.csv')):
    df_ant = pd.read_csv('evo_last_data.csv')
    # Evita acumulação de veículos desaparecidos repetidamente
    df_ant = df_ant[df_ant['Status_tag'] != 'start']

In [ ]:
try:
    i = 0
    response = None
    # Loop para evitar erros de requisições demoradas
    while (response == None and i < 5):
        # Fazendo uma requisição a api da evo e formantando como json
        response = urlopen('https://www.evo.ca/api/Cars.aspx', timeout=20).read().decode('UTF-8')
        i += 1
except Exception as e:
    print('Erro na requisição base: ' + str(e))
    return

if (response == None):
    return

json_data = json.loads(response)

# Colocando os dados em um dataframe
df_atual = pd.DataFrame.from_dict(json_data['data'])

## Adicionando a hora da coleta

In [ ]:
# Obtendo a hora em que foi coletado

import datetime as dt
import pytz

# Coletando a hora atual de vancouver
vanc_tz = pytz.timezone('America/Vancouver')
time = dt.datetime.now(tz=vanc_tz)

cap_time = []

for i in range(len(df_atual)):
    cap_time.append(time)
    
cap_time = pd.DataFrame(cap_time, columns=['Capture_time'])

df_atual = pd.concat([cap_time, df_atual], axis=1)

## Adicionando tags de estado

In [ ]:
# Adicionando um coluna para informar o status do veículo como parked, start ou end
# a fim de analisar o tempo de viagem
status_tag = []

for i in range(len(df_atual)):
    status_tag.append('parked')
    
status_tag = pd.DataFrame(status_tag, columns=['Status_tag'])

df_atual = pd.concat([df_atual, status_tag], axis=1)

In [ ]:
# A primeira execução não tem ainda o df_ant
if (exists('evo_last_data.csv')):
    for i in range(len(df_ant)):
        # Se um carro que estava registrado no minuto anterior sumiu, ele iniciou uma viagem
        if (df_ant['Id'].iloc[i] not in df_atual['Id'].values):
        
            missed_car = df_ant.iloc[i]
            
            # Erro ocasionado pelo fato de que a primeira execução não detém da coluna Status tag
            try:
                missed_car['Status_tag'] = 'start'
                missed_car['Capture_time'] = time
            except:        
                # Criando uma nova series para ser adicionada ao fim do dataframe atual
                missed_car = pd.concat([missed_car, pd.Series(['start'], dtype=str)], axis=0)
                missed_car.rename(index={0 : 'Status_tag'}, inplace=True)

            df_atual = df_atual.append(missed_car, ignore_index=True)

    for i in range(len(df_atual)):
        # Se um carro não estava registrado no minuto anterior e está no atual, então ele terminou uma viagem
        if (df_atual['Id'].iloc[i] not in df_ant['Id'].values):
            df_atual['Status_tag'].iloc[i] = 'end'
else:
    pass

In [ ]:
# Verificando se já existe o arquivo, evita a repetição dos labels das colunas
# Atualiza o csv de acumulo de dados e salva um para comparação como minuto anterior
if (exists('evo_data.csv')):
    df_atual.to_csv('evo_data.csv', mode='a', header=False, index=False, encoding='utf-8')
    df_atual.to_csv('evo_last_data.csv', index=False, encoding='utf-8')
else:
    df_atual.to_csv('evo_data.csv', index=False, encoding='utf-8')
    df_atual.to_csv('evo_last_data.csv', index=False, encoding='utf-8')

# Coletando o tempo de trajeto Google API e real

In [ ]:
aux_missed = df_atual[df_atual['Status_tag'] == 'start']

# Lendo csv com dados dos carros que iniciaram viagens desde o minuto anterior
if (exists('missed_cars.csv')):
    df_missed = pd.read_csv('missed_cars.csv')
    
    if (len(aux_missed) > 0):
        # Atualizando os dados com os carros perdidos do minuto atual
        df_missed = pd.concat([df_missed, aux_missed], axis=0)
    
    # consertando os indices
    df_missed.index = range(len(df_missed))

elif (len(aux_missed) > 0):
    aux_missed.to_csv('missed_cars.csv', index=False, encoding='utf-8')
    df_missed = aux_missed
    
    # consertando os indices
    df_missed.index = range(len(df_missed))

In [ ]:
# Veículos que terminaram a sua viagem no minuto atual
df_end = df_atual[df_atual['Status_tag'] == 'end']

In [ ]:
# Faz a diferença entre duas horas dadas e retorna em minutos
def Hour_Diff(h1,h2):
    h1Aux = dt.datetime.strptime(h1,"%Y-%m-%d %H:%M:%S.%f-08:00")
    h2Aux = dt.datetime.strptime(h2,"%Y-%m-%d %H:%M:%S.%f-08:00")
    diff = abs((h1Aux - h2Aux)).total_seconds()/60
    
    return diff

In [ ]:
# Só analisa os tempos de viagem se encontrar algum veículo que terminou a viagem
try:
    if (len(df_end) > 0):     
        i = 0
        while (i < len(df_missed)):
            if (df_missed['Id'].iloc[i] in df_end['Id'].values):
                
                start = df_missed.iloc[i]
                end = df_end[df_end['Id'] == df_missed['Id'].iloc[i]]
                end_lat = str(end.get_value(index=end.index[0], col='Lat'))
                end_lon = str(end.get_value(index=end.index[0], col='Lon'))
                end_time = str(end.get_value(index=end.index[0], col='Capture_time'))
                end_fuel = int(end.get_value(index=end.index[0], col='Fuel'))

                
                test = True
                maps_time = None
                
                # Evita requisições desnecessárias de carros que retornam ao mesmo ponto
                if (str(start['Lat']) == end_lat and str(start['Lon']) == end_lon):
                    maps_time = 0
                    test = False
                
                # Loop para trocar keys assim que acabar o limite de requisições de cada key
                while(test and key_index < len(KEYs)):
                    try:
                        # Requisição a Google API para tirar o tempo usual de trajeto
                        maps_req = urlopen('https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&origins='+str(start['Lat'])+','+str(start['Lon'])+
                                            '&destinations='+end_lat+','+end_lon+'&key='+KEYs[key_index]).read().decode('UTF-8')
                        
                        json_maps = json.loads(maps_req)
                        
                        # Dividindo por 60 para ter o tempo em minutos
                        maps_time = float(json_maps['rows'][0]['elements'][0]['duration']['value'])/60
                        
                        test = False
                    except Exception as e:
                        print(str(e) + ' index key: ' + str(key_index))
                        key_index += 1
                        test = True
                
                # Analise do limite de keys e se a api retornou algum valor válido
                if (key_index == len(KEYs)):
                    print('Terminou as keys')
                if (maps_time == None):
                    print('Erro na requisição do maps')
                    continue

                # Fazendo o tempo de duração efetuado pelo motorista
                real_time = Hour_Diff(start['Capture_time'], end_time)

                data = [{'Start_time' : start['Capture_time'], 'End_time' : end_time, 'Id' : start['Id'],
                        'Start_lat' : start['Lat'], 'Start_lon' : start['Lon'], 'End_lat' : end_lat,
                        'End_lon' : end_lon, 'Maps_duration' : maps_time, 'Real_duration' : real_time,
                        'Fuel_start' : start['Fuel'], 'Fuel_end': end_fuel}]

                try:
                    df_travels = df_travels.append(data, ignore_index=True)
                except:
                    df_travels = pd.DataFrame(data, index=[0])
                
                # Reorganizando as colunas
                df_travels = df_travels[['Start_time', 'End_time', 'Id', 'Start_lat', 'Start_lon', 'End_lat',
                                         'End_lon', 'Maps_duration', 'Real_duration', 'Fuel_start', 'Fuel_end']]

                # Retira do data frame o carro encontrado e conserta o indice
                df_missed = df_missed.drop([i])
                df_missed.index = range(len(df_missed))
                i -= 1
            
            # passo do loop
            i += 1
        
        
except Exception as e:
    print('Missed Loop: '+str(e))
    pass
    

In [ ]:
# Salvando o registro de viagens
try:
    if (exists('travels.csv')):
        df_travels.to_csv('travels.csv', mode='a', header=False, index=False, encoding='utf-8')
    else:
        df_travels.to_csv('travels.csv', index=False, encoding='utf-8')
except Exception as e:
    print('Nao encontrou viagens: '+str(e))

In [ ]:
# Salvando os veículos que ainda estão em viagem
try:
    if (exists('missed_cars.csv') and len(df_missed) > 0):
        df_missed.to_csv('missed_cars.csv', index=False, encoding='utf-8')
except Exception as e:
    print('Missed save: ' + str(e))
    pass